# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arenaコンペティションにおける人間の好み予測に関連する問題に取り組んでいます。具体的には、ユーザーが2つのチャットボットからの応答のどちらを好むかを予測するための機械学習モデルを構築しています。このプロジェクトでは、以下の手法やライブラリが使用されています。

1. **主要なライブラリ**:
   - **Transformers**: 事前学習済みのGemmaモデルを使用するために必要です。また、トークナイザーやトレーニングの設定を行うための機能も提供します。
   - **Datasets**: データセットの操作を行います。
   - **PEFT**: パラメータ効率的なファインチューニングを行うためにLoRA（低ランクアダプター）を設定します。
   - **Scikit-learn**: モデルの評価に使用されるメトリクスを計算するために利用されます。

2. **問題解決のアプローチ**:
   - ノートブックはまず、Gemmaモデルのインストールと必要なライブラリのインポートから始まります。そして、トレーニングや評価に必要な設定を一元管理するためのデータクラス`Config`を定義します。
   - LoRAの設定が行われ、特定のモデルモジュールへの適用が定義されています。これにより、トレーニングの効率を高めることが可能になります。
   - カスタムトークナイザーを用いて、データセットからの入力を適切に処理し、ユーザーの好みに基づいたラベルを生成します。
   - モデルのトレーニングは、データセットを5つの分割にして、交差検証を通じて行われます。この過程で、評価メトリクスとしてロス（log-loss）や精度（accuracy）を計算し、モデルの性能を評価します。

全体として、このノートブックは、深層学習を用いてチャットボットの応答に対するユーザーの好みを予測するためのフレームワークを構築することを目的としており、GemmaモデルのLoRAによるファインチューニング技術を活用しています。

---


# 用語概説 
以下は、与えられたJupyter Notebookに登場する専門用語の簡単な解説です。初心者の方がつまずく可能性がある比較的マイナーな用語や、このノートブック特有の知識に焦点を当てています。

1. **gemma-2**: 大規模言語モデルの一つで、特にシーケンス分類タスク向けに設計されています。ハイパフォーマンスのための特別なアーキテクチャが組み込まれています。

2. **BitsAndBytesConfig**: メモリ効率を考慮した設定で、特に量子化技術を用いて、モデルのサイズを小さく保ちながらパフォーマンスを向上させるための設定です。

3. **Gradients Accumulation**: バッチサイズを実質的に増やすために、小さいバッチで計算した勾配を複数回蓄積し、指定したステップ数後にモデルの重みを更新する技術です。

4. **LoraConfig**: LoRA（Low-Rank Adaptation）に基づく設定構造で、効率的なファインチューニングを行うための様々なパラメータを含んでいます。LoRAは、モデルの特定機能を効率的に調整するために低ランクのパラメータを用います。

5. **self-attention**: ニューラルネットワークの一種において、入力の異なる部分がどのように相互作用するかを学習する仕組みです。特に、トランスフォーマーモデルで重要な役割を果たします。

6. **Data Collator**: データをバッチ単位に整形するためのツールで、パディングやフォーマットを統一するために用います。これにより、異なる長さのシーケンスでも一貫性感を持たせることができます。

7. **TaskType**: タスクの種類を定義する列挙型で、ここではシーケンス分類(`SEQ_CLS`)を指定しています。モデルがどのようなタスクを実行するかを明示的に設定します。

8. **Softmax**: 出力ベクトルを確率分布に変換するための関数で、主に多クラス分類問題に使用されます。各クラスに対するスコアを確率として解釈します。

9. **log loss**: バイナリまたは多クラス分類タスクで用いられる損失関数の一つで、予測確率と実際のラベルとの不一致を測定します。モデルの性能を評価するのに使われる指標です。

これらの用語を理解することで、Jupyter Notebook内のコードやコメントの意味がより明確になるでしょう。

---


In [ ]:
# gemma-2は transformers>=4.42.3 から利用可能です
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

In [ ]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

# 必要なライブラリをインポートします。
# os, copy: ファイル操作やオブジェクトのコピーに使用されます。
# dataclass: データクラスを作成するために使います。
# numpy: 数値演算用のライブラリです。
# torch: PyTorchライブラリで、深層学習モデルの構築に使用します。
# datasets: データセットの操作に使用します。
# transformers: 事前学習済みの変換器モデルを扱うための機能群です。
# peft: パラメータ効率的ファインチューニングに関連する機能を提供します。
# sklearn.metrics: メトリクスの計算に利用します。

### 設定


In [ ]:
@dataclass
class Config:
    output_dir: str = "output"
    checkpoint: str = "/kaggle/input/checkpoint-5200/checkpoint-5200"  # 4ビット量子化されたgemma-2-9b-instruct
    max_length: int = 1024
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 2
    gradient_accumulation_steps: int = 2  # グローバルバッチサイズは8です
    per_device_eval_batch_size: int = 8
    n_epochs: int = 1
    freeze_layers: int = 16  # 合計42層のうち、最初の16層にはアダプターを追加しません
    lr: float = 2e-4
    warmup_steps: int = 20
    lora_r: int = 16
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = "none"
    
config = Config()

# 設定用のデータクラスConfigを定義します。
# 各属性にデフォルト値を設定し、モデルのトレーニングや評価に使用します。

#### トレーニング引数


In [ ]:
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="steps",
    save_steps=200,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
)

# トレーニング時の引数を設定するTrainingArgumentsを作成します。
# 各パラメータはモデルのトレーニングにどのように影響を与えるかを指定します。

#### LoRA設定


In [ ]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    # 自己注意のみにターゲットを絞る
    target_modules=["q_proj", "k_proj", "v_proj"],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

# LoRA(低ランクアダプター)の設定を定義します。
# モデルの特定のモジュールに適用し、トレーニングの効率を高めます。

### トークナイザーとモデルのインスタンス化


In [ ]:
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True  # 終端記号<eos>を追加します
tokenizer.padding_side = "right"

# Gemmaトークナイザーを事前学習済みのチェックポイントからロードし、
# 必要な設定を行います。

In [ ]:
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model

# モデルを事前学習済みのチェックポイントからロードし、
# シーケンス分類用に構成し、LoRAによるファインチューニングの準備をします。

In [ ]:
model.print_trainable_parameters()

### データセットのインスタンス化


In [ ]:
ds = Dataset.from_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
#ds = ds.select(torch.arange(100))  # デモ目的で最初の100件のみ使用します

In [ ]:
ds

In [ ]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __call__(self, batch: dict) -> dict:
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:
                label = 0
            elif b_win:
                label = 1
            else:
                label = 2
            labels.append(label)
        return {**tokenized, "labels": labels}
        
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))

# カスタムトークナイザーを定義します。
# トークナイザーを使って入力データを処理し、ラベルを生成します。

In [ ]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)

### メトリクスの計算

LBで使用されるロス（log-loss）と補助的メトリクスとしての精度（accuracy）を計算します。


In [ ]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

# 評価予測からメトリクス（精度とロス）を計算します。
# softmaxを使って確率を計算し、評価指標を返します。

### 分割

ここでは、trainとevalを`id % 5`に従って分割します。


In [ ]:
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
    ) 
    for fold_idx in range(config.n_splits)
]

# データセットを5分割し、各分割にインデックスを割り当てます。
# トレーニング用インデックスと評価用インデックスを生成します。

In [ ]:
train_idx, eval_idx = folds[config.fold_idx]

trainer = Trainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train(resume_from_checkpoint="/kaggle/input/checkpoint-5200/checkpoint-5200")

# Trainerをインスタンス化し、トレーニングを行います。
# 前回のチェックポイントから再開することも可能です。